In [60]:
# Download the dataset from Kaggle using the Kaggle API

import opendatasets as od

# Eric's Kaggle API key
# "username":"ericthedataguy",
# "key":"875c0de0faea2fefa081c78eb470a347"

url_cleaned = 'https://www.kaggle.com/datasets/ericthedataguy/cleaned-hotel-review-for-ba820'

od.download(url_cleaned)

Skipping, found downloaded files in "./cleaned-hotel-review-for-ba820" (use force=True to force download)


In [61]:
# Read the dataset into a pandas dataframe

import pandas as pd

filepath = '/Users/hakukazuho/Desktop/Luxurious-Hotel-Review/cleaned-hotel-review-for-ba820/hotel_reviews_cleaned.csv'
df = pd.read_csv(filepath)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [156]:
df['Tags'].head()

0                                                            [' Leisure trip ', ' Couple ', ' Duplex Double Room ', ' Stayed 6 nights ']
1                                                            [' Leisure trip ', ' Couple ', ' Duplex Double Room ', ' Stayed 4 nights ']
2    [' Leisure trip ', ' Family with young children ', ' Duplex Double Room ', ' Stayed 3 nights ', ' Submitted from a mobile device ']
3                                                     [' Leisure trip ', ' Solo traveler ', ' Duplex Double Room ', ' Stayed 3 nights ']
4                                     [' Leisure trip ', ' Couple ', ' Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']
Name: Tags, dtype: object

In [62]:
# Drop outliers in the 'Reviewer_Score' column

# Calculate the IQR
Q1 = df['Reviewer_Score'].quantile(0.25)
Q3 = df['Reviewer_Score'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Drop the outliers and store the results in df_drop
df_drop = df[(df['Reviewer_Score'] >= lower_bound) & (df['Reviewer_Score'] <= upper_bound)]
df_drop.shape[0]

499831

In [63]:
test = df_drop.iloc[:1000]
test.to_csv('test.csv', index=False)

In [64]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1034
Data columns (total 17 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Hotel_Address                               1000 non-null   object 
 1   Additional_Number_of_Scoring                1000 non-null   int64  
 2   Review_Date                                 1000 non-null   object 
 3   Average_Score                               1000 non-null   float64
 4   Hotel_Name                                  1000 non-null   object 
 5   Reviewer_Nationality                        1000 non-null   object 
 6   Negative_Review                             1000 non-null   object 
 7   Review_Total_Negative_Word_Counts           1000 non-null   int64  
 8   Total_Number_of_Reviews                     1000 non-null   int64  
 9   Positive_Review                             1000 non-null   object 
 10  Review_Total_Posi

In [65]:
col_list = ['Hotel_Name', 'Positive_Review']

test_subset = test[col_list]
test_subset['Positive_Review'] = test_subset['Positive_Review'].str.lower()
test_subset.head()

/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/2274795498.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_subset['Positive_Review'] = test_subset['Positive_Review'].str.lower()


,Hotel_Name,Positive_Review
1,Hotel Arena,no real complaints the hotel was great great location surroundings rooms amenities and service two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one bit confusing secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great aside from those minor minor things fantastic spot and will be back when i return to amsterdam
2,Hotel Arena,location was good and staff were ok it is cute hotel the breakfast range is nice will go back
4,Hotel Arena,amazing location and building romantic setting
5,Hotel Arena,good restaurant with modern design great chill out place great park nearby the hotel and awesome main stairs
6,Hotel Arena,the room is spacious and bright the hotel is located in a quiet and beautiful park


In [66]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.util import ngrams

# Drop stop words
stop_words = set(stopwords.words('english'))
test_subset['PR_Drop'] = test_subset['Positive_Review'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Tokenize using 1-gram
test_subset['PR_T_3g'] = test_subset['Positive_Review'].apply(lambda x: list(ngrams(x.split(), 3)))

# Tokenize using 3-grams
test_subset['PR_D_T_3g'] = test_subset['PR_Drop'].apply(lambda x: list(ngrams(x.split(), 3)))
test_subset.head()

/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/1812365478.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_subset['PR_Drop'] = test_subset['Positive_Review'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/1812365478.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_subset['PR_T_3g'] = test_subset['Positive_Review'].apply(lambda x: list(ngrams(x.split(), 3)))
/var/fold

,Hotel_Name,Positive_Review,PR_Drop,PR_T_3g,PR_D_T_3g
1,Hotel Arena,no real complaints the hotel was great great location surroundings rooms amenities and service two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one bit confusing secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great aside from those minor minor things fantastic spot and will be back when i return to amsterdam,real complaints hotel great great location surroundings rooms amenities service two recommendations however firstly staff upon check confusing regarding deposit payments staff offer upon checkout refund original payment make new one bit confusing secondly site restaurant bit lacking well thought excellent quality food anyone vegetarian vegan background even wrap toasted sandwich option would great aside minor minor things fantastic spot back return amsterdam,"[(no, real, complaints), (real, complaints, the), (complaints, the, hotel), (the, hotel, was), (hotel, was, great), (was, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, and), (amenities, and, service), (and, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, the), (firstly, the, staff), (the, staff, upon), (staff, upon, check), (upon, check, in), (check, in, are), (in, are, very), (are, very, confusing), (very, confusing, regarding), (confusing, regarding, deposit), (regarding, deposit, payments), (deposit, payments, and), (payments, and, the), (and, the, staff), (the, staff, offer), (staff, offer, you), (offer, you, upon), (you, upon, checkout), (upon, checkout, to), (checkout, to, refund), (to, refund, your), (refund, your, original), (your, original, payment), (original, payment, and), (payment, and, you), (and, you, can), (you, can, make), (can, make, a), (make, a, new), (a, new, one), (new, one, bit), (one, bit, confusing), (bit, confusing, secondly), (confusing, secondly, the), (secondly, the, on), (the, on, site), (on, site, restaurant), (site, restaurant, is), (restaurant, is, a), (is, a, bit), (a, bit, lacking), (bit, lacking, very), (lacking, very, well), (very, well, thought), (well, thought, out), (thought, out, and), (out, and, excellent), (and, excellent, quality), (excellent, quality, food), (quality, food, for), (food, for, anyone), (for, anyone, of), (anyone, of, a), (of, a, vegetarian), (a, vegetarian, or), (vegetarian, or, vegan), (or, vegan, background), (vegan, background, but), (background, but, even), (but, even, a), (even, a, wrap), (a, wrap, or), (wrap, or, toasted), (or, toasted, sandwich), (toasted, sandwich, option), (sandwich, option, would), (option, would, be), (would, be, great), (be, great, aside), (great, aside, from), (aside, from, those), (from, those, minor), (those, minor, minor), (minor, minor, things), (minor, things, fantastic), (things, fantastic, spot), (fantastic, spot, and), (spot, and, will), (and, will, be), (will, be, back), (be, back, when), (back, when, i), (when, i, return), (i, return, to), ...]","[(real, complaints, hotel), (complaints, hotel, great), (hotel, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, service), (amenities, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, staff), (firstly, staff, upon), (staff, upon, check), (upon, check, confusing), (check, confusing, regarding), (confusing, regarding, deposit), (regarding, deposit, payments), (deposit, payme

In [67]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Create an instance of the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Function to check if a token has positive sentiment
def has_positive_sentiment(token):
    sentiment_score = sia.polarity_scores(' '.join(token))
    return sentiment_score['compound'] > 0

# Apply sentiment analysis and filter tokens with positive sentiment
test_subset['Positive_Tokens'] = test_subset['PR_D_T_3g'].apply(lambda x: [token for token in x if has_positive_sentiment(token)])

# Drop tokens with no positive sentiment
test_subset['Positive_Tokens'] = test_subset['Positive_Tokens'].apply(lambda x: x if len(x) > 0 else None)

# Drop rows with no positive tokens
test_subset = test_subset.dropna(subset=['Positive_Tokens'])

test_subset.head()


/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/3522127624.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_subset['Positive_Tokens'] = test_subset['PR_D_T_3g'].apply(lambda x: [token for token in x if has_positive_sentiment(token)])
/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/3522127624.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_subset['Positive_Tokens'] = test_subset['Positive_Tokens'].apply(lambda x: x if len(x) > 0 else None)


,Hotel_Name,Positive_Review,PR_Drop,PR_T_3g,PR_D_T_3g,Positive_Tokens
1,Hotel Arena,no real complaints the hotel was great great location surroundings rooms amenities and service two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one bit confusing secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great aside from those minor minor things fantastic spot and will be back when i return to amsterdam,real complaints hotel great great location surroundings rooms amenities service two recommendations however firstly staff upon check confusing regarding deposit payments staff offer upon checkout refund original payment make new one bit confusing secondly site restaurant bit lacking well thought excellent quality food anyone vegetarian vegan background even wrap toasted sandwich option would great aside minor minor things fantastic spot back return amsterdam,"[(no, real, complaints), (real, complaints, the), (complaints, the, hotel), (the, hotel, was), (hotel, was, great), (was, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, and), (amenities, and, service), (and, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, the), (firstly, the, staff), (the, staff, upon), (staff, upon, check), (upon, check, in), (check, in, are), (in, are, very), (are, very, confusing), (very, confusing, regarding), (confusing, regarding, deposit), (regarding, deposit, payments), (deposit, payments, and), (payments, and, the), (and, the, staff), (the, staff, offer), (staff, offer, you), (offer, you, upon), (you, upon, checkout), (upon, checkout, to), (checkout, to, refund), (to, refund, your), (refund, your, original), (your, original, payment), (original, payment, and), (payment, and, you), (and, you, can), (you, can, make), (can, make, a), (make, a, new), (a, new, one), (new, one, bit), (one, bit, confusing), (bit, confusing, secondly), (confusing, secondly, the), (secondly, the, on), (the, on, site), (on, site, restaurant), (site, restaurant, is), (restaurant, is, a), (is, a, bit), (a, bit, lacking), (bit, lacking, very), (lacking, very, well), (very, well, thought), (well, thought, out), (thought, out, and), (out, and, excellent), (and, excellent, quality), (excellent, quality, food), (quality, food, for), (food, for, anyone), (for, anyone, of), (anyone, of, a), (of, a, vegetarian), (a, vegetarian, or), (vegetarian, or, vegan), (or, vegan, background), (vegan, background, but), (background, but, even), (but, even, a), (even, a, wrap), (a, wrap, or), (wrap, or, toasted), (or, toasted, sandwich), (toasted, sandwich, option), (sandwich, option, would), (option, would, be), (would, be, great), (be, great, aside), (great, aside, from), (aside, from, those), (from, those, minor), (those, minor, minor), (minor, minor, things), (minor, things, fantastic), (things, fantastic, spot), (fantastic, spot, and), (spot, and, will), (and, will, be), (will, be, back), (be, back, when), (back, when, i), (when, i, return), (i, return, to), ...]","[(real, complaints, hotel), (complaints, hotel, great), (hotel, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, service), (amenities, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, staff), (firstly, staff, upon), (staff, upon, check), (upon, check, confusing), (check, confusing, regarding), (confusing, regarding, deposit), (regarding, deposit, payments),

In [68]:
import nltk

test_subset['positive_words'] = test_subset['Positive_Tokens'].apply(lambda x: [word for token in x for word in token])
test_subset.head()

# Function to extract nouns/subjects from a list of words
def extract_nouns(words):
    tagged_words = nltk.pos_tag(words)
    nouns = [word for word, pos in tagged_words if pos.startswith('NN')]
    return nouns

# Apply the function to extract nouns/subjects and create the "Positive_Tags" column
test_subset['Positive_Tags'] = test_subset['positive_words'].apply(extract_nouns)

# Remove duplicate values in each row of the "Positive_Tags" column
test_subset['Positive_Tags'] = test_subset['Positive_Tags'].apply(lambda x: list(set(x)) if x is not None else None)

test_subset.head()


,Hotel_Name,Positive_Review,PR_Drop,PR_T_3g,PR_D_T_3g,Positive_Tokens,positive_words,Positive_Tags
1,Hotel Arena,no real complaints the hotel was great great location surroundings rooms amenities and service two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one bit confusing secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great aside from those minor minor things fantastic spot and will be back when i return to amsterdam,real complaints hotel great great location surroundings rooms amenities service two recommendations however firstly staff upon check confusing regarding deposit payments staff offer upon checkout refund original payment make new one bit confusing secondly site restaurant bit lacking well thought excellent quality food anyone vegetarian vegan background even wrap toasted sandwich option would great aside minor minor things fantastic spot back return amsterdam,"[(no, real, complaints), (real, complaints, the), (complaints, the, hotel), (the, hotel, was), (hotel, was, great), (was, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, and), (amenities, and, service), (and, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, the), (firstly, the, staff), (the, staff, upon), (staff, upon, check), (upon, check, in), (check, in, are), (in, are, very), (are, very, confusing), (very, confusing, regarding), (confusing, regarding, deposit), (regarding, deposit, payments), (deposit, payments, and), (payments, and, the), (and, the, staff), (the, staff, offer), (staff, offer, you), (offer, you, upon), (you, upon, checkout), (upon, checkout, to), (checkout, to, refund), (to, refund, your), (refund, your, original), (your, original, payment), (original, payment, and), (payment, and, you), (and, you, can), (you, can, make), (can, make, a), (make, a, new), (a, new, one), (new, one, bit), (one, bit, confusing), (bit, confusing, secondly), (confusing, secondly, the), (secondly, the, on), (the, on, site), (on, site, restaurant), (site, restaurant, is), (restaurant, is, a), (is, a, bit), (a, bit, lacking), (bit, lacking, very), (lacking, very, well), (very, well, thought), (well, thought, out), (thought, out, and), (out, and, excellent), (and, excellent, quality), (excellent, quality, food), (quality, food, for), (food, for, anyone), (for, anyone, of), (anyone, of, a), (of, a, vegetarian), (a, vegetarian, or), (vegetarian, or, vegan), (or, vegan, background), (vegan, background, but), (background, but, even), (but, even, a), (even, a, wrap), (a, wrap, or), (wrap, or, toasted), (or, toasted, sandwich), (toasted, sandwich, option), (sandwich, option, would), (option, would, be), (would, be, great), (be, great, aside), (great, aside, from), (aside, from, those), (from, those, minor), (those, minor, minor), (minor, minor, things), (minor, things, fantastic), (things, fantastic, spot), (fantastic, spot, and), (spot, and, will), (and, will, be), (will, be, back), (be, back, when), (back, when, i), (when, i, return), (i, return, to), ...]","[(real, complaints, hotel), (complaints, hotel, great), (hotel, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, service), (amenities, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, staff), (firstly, staff, upon), (staff, upon, check), (upon, check, confusing), (check, confusing, regarding), (confusing, regarding, deposit), (r

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizer object
vectorizer = TfidfVectorizer()

# Fit and transform the Positive_Tags column using TF-IDF
tfidf_matrix = vectorizer.fit_transform(test_subset['Positive_Tags'].apply(lambda x: ' '.join(x) if x is not None else ''))

# Get the feature names (tags)
feature_names = vectorizer.get_feature_names_out()

# Create an empty list to store the important tags for each hotel
important_tags = []

# Iterate over each row in the tfidf_matrix
for i in range(tfidf_matrix.shape[0]):
    # Get the TF-IDF scores for the current row
    row = tfidf_matrix.getrow(i).toarray()[0]
    
    # Sort the scores in descending order and get the indices of the top 5 tags
    top_tags_indices = row.argsort()[::-1][:5]
    
    # Get the top 5 tags from the feature_names list
    top_tags = [feature_names[idx] for idx in top_tags_indices]
    
    # Append the top tags to the important_tags list
    important_tags.append(top_tags)

# Add the important_tags list as a new column in test_subset
test_subset['Important_Tags'] = important_tags

test_subset.head()


,Hotel_Name,Positive_Review,PR_Drop,PR_T_3g,PR_D_T_3g,Positive_Tokens,positive_words,Positive_Tags,Important_Tags
1,Hotel Arena,no real complaints the hotel was great great location surroundings rooms amenities and service two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one bit confusing secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great aside from those minor minor things fantastic spot and will be back when i return to amsterdam,real complaints hotel great great location surroundings rooms amenities service two recommendations however firstly staff upon check confusing regarding deposit payments staff offer upon checkout refund original payment make new one bit confusing secondly site restaurant bit lacking well thought excellent quality food anyone vegetarian vegan background even wrap toasted sandwich option would great aside minor minor things fantastic spot back return amsterdam,"[(no, real, complaints), (real, complaints, the), (complaints, the, hotel), (the, hotel, was), (hotel, was, great), (was, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, and), (amenities, and, service), (and, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, the), (firstly, the, staff), (the, staff, upon), (staff, upon, check), (upon, check, in), (check, in, are), (in, are, very), (are, very, confusing), (very, confusing, regarding), (confusing, regarding, deposit), (regarding, deposit, payments), (deposit, payments, and), (payments, and, the), (and, the, staff), (the, staff, offer), (staff, offer, you), (offer, you, upon), (you, upon, checkout), (upon, checkout, to), (checkout, to, refund), (to, refund, your), (refund, your, original), (your, original, payment), (original, payment, and), (payment, and, you), (and, you, can), (you, can, make), (can, make, a), (make, a, new), (a, new, one), (new, one, bit), (one, bit, confusing), (bit, confusing, secondly), (confusing, secondly, the), (secondly, the, on), (the, on, site), (on, site, restaurant), (site, restaurant, is), (restaurant, is, a), (is, a, bit), (a, bit, lacking), (bit, lacking, very), (lacking, very, well), (very, well, thought), (well, thought, out), (thought, out, and), (out, and, excellent), (and, excellent, quality), (excellent, quality, food), (quality, food, for), (food, for, anyone), (for, anyone, of), (anyone, of, a), (of, a, vegetarian), (a, vegetarian, or), (vegetarian, or, vegan), (or, vegan, background), (vegan, background, but), (background, but, even), (but, even, a), (even, a, wrap), (a, wrap, or), (wrap, or, toasted), (or, toasted, sandwich), (toasted, sandwich, option), (sandwich, option, would), (option, would, be), (would, be, great), (be, great, aside), (great, aside, from), (aside, from, those), (from, those, minor), (those, minor, minor), (minor, minor, things), (minor, things, fantastic), (things, fantastic, spot), (fantastic, spot, and), (spot, and, will), (and, will, be), (will, be, back), (be, back, when), (back, when, i), (when, i, return), (i, return, to), ...]","[(real, complaints, hotel), (complaints, hotel, great), (hotel, great, great), (great, great, location), (great, location, surroundings), (location, surroundings, rooms), (surroundings, rooms, amenities), (rooms, amenities, service), (amenities, service, two), (service, two, recommendations), (two, recommendations, however), (recommendations, however, firstly), (however, firstly, staff), (firstly, staff, upon), (staff, upon, check), (upon, check, confusing), (check, confusing, regarding), (confusing, regardin

In [70]:
tag_table = test_subset.groupby('Hotel_Name')['Important_Tags'].sum().sort_values(ascending=False).reset_index()
tag_table.head()

,Hotel_Name,Important_Tags
0,Apex Temple Court Hotel,"[requests, choices, bed, zone, experience, trains, check, attractions, walk, rooms, value, room, evaluate, exhibition, executive, standard, rooms, location, zone, evening, backdrop, city, walk, beautiful, evenings, excellent, room, location, zone, evening, novel, francesco, questions, idea, cute, bathroom, room, zone, evenings, experience, walk, shower, excellent, bathroom, bed, fleet, concierge, stations, minutes, stay, spaciousness, standard, hotel, zone, espresso, stay, clean, room, location, zone, cosy, space, cool, lobby, bar, conceierge, francis, thanks, service, zone, relaxing, stay, room, hotel, location, presentation, room, hotel, zone, evening, outage, system, kind, shower, excellent, club, lounge, comfy, bed, room, service, excellent, bathroom, room, location, covent, couch, westminster, reach, bath, ...]"
1,Hotel Arena,"[complaints, payment, option, refund, things, range, cute, breakfast, hotel, location, setting, building, location, zone, experience, chill, stairs, awesome, design, place, beautiful, park, room, hotel, evening, set, quality, food, park, breakfast, morning, walk, service, food, park, inviting, couples, beauty, kept, pictures, comfy, bed, location, evenings, experience, appreciation, care, structure, price, room, historic, building, zone, evenings, experience, awesome, floor, minutes, cool, tram, gin, onsite, bobby, cafe, building, system, square, dam, tram, hotel, window, areas, nice, building, room, girls, quirky, hotels, history, breakfast, oostpark, yards, continental, selection, area, locates, need, interior, design, beautiful, locations, people, way, help, city, difficulty, shampoo, double, shops, oosterpark, ...]"
2,K K Hotel George,"[absolute, delight, smart, facilities, shower, daytime, staff, zone, exhibition, executive, term, recommendation, friend, evenings, corner, ventilation, division, fan, super, london, deal, everyone, plenty, choices, gardens, sunshine, order, guests, drink, restaurants, someone, kind, choice, reception, clean, pleasant, restaurant, breakfast, staff, zone, enjoy, feature, secure, parking, time, mirror, buying, windows, billowing, hair, gardens, zone, evenings, experience, exhibition, evaluate, veeery, promotion, pay, future, gardens, comfy, beds, staff, evening, visit, way, get, clean, hotel, linens, bathroom, room, evenings, experience, inside, glass, beer, relax, feel, quiet, buffet, breakfast, location, zone, return, time, day, street, buffet, seat, position, helpful, garden, breakfast, bustle, restaurants, location, evenings, experience, ...]"


In [71]:
from ipywidgets import interact, Dropdown

def generate_word_cloud():
    # Create a dropdown menu to select the hotel name
    hotel_dropdown = Dropdown(options=test_subset['Hotel_Name'].unique(), description='Hotel Name:')

    # Define a function to generate the word cloud
    def generate_cloud(hotel_name):
        # Filter the dataframe based on the hotel name
        hotel_df = test_subset[test_subset['Hotel_Name'] == hotel_name]

        # Combine all the positive tags into a single string
        tags_text = ' '.join(hotel_df['Important_Tags'].sum())

        # Create a word cloud object
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(tags_text)

        # Generate the title for the word cloud
        title = f"The Top Tags of {hotel_name}"

        # Display the word cloud with the title
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(title)
        plt.axis('off')
        plt.show()

    # Interact the dropdown menu with the generate_cloud function
    interact(generate_cloud, hotel_name=hotel_dropdown)


In [72]:
generate_word_cloud()

interactive(children=(Dropdown(description='Hotel Name:', options=('Hotel Arena', 'K K Hotel George', 'Apex Te…

A brief summary of the text-to-tags transformation:

1. drop stop words (english) from original positive tags

2. use 3-grams to tokenize dropped reviews

3. conduct sentiment analysis for each token, if positive sentiment, add to positive token

4. flatten out positive token and extract noun to positive tags

5. use tf-idf to only keep those who are important in important tags

Current problem & future plan:

1. still some non-noun in the tag column

2. build a pipeline for preprocessing

3. check manually & randomly to test tag extraction accuracy.

In [134]:
import nltk
import pandas as pd

# Set the display option to show full text
pd.set_option('display.max_colwidth', None)

# Randomly select 5 rows from the test_subset dataframe
random_rows = test.sample(n=5, random_state=42)

# Display the Positive_Review and Important_Tags columns with the full text
random_rows['Positive_Review']

# Function to extract keywords/tags from a positive review
def extract_keywords(review):
    # Tokenize the review into individual words
    tokens = nltk.word_tokenize(review)
    
    # Perform part-of-speech tagging on the tokens
    tagged_tokens = nltk.pos_tag(tokens)
    
    # Extract nouns and adjectives as keywords/tags
    keywords = []
    noun = None
    for word, pos in tagged_tokens:
        if pos.startswith('NN'):  # Look for nouns
            noun = word
        elif pos.startswith('JJ') and noun:  # Look for adjectives describing nouns
            keywords.append(f"{word} {noun}")
            noun = None
    
    return keywords

# Apply the extract_keywords function to the Positive_Review column
test['Keywords'] = test['Positive_Review'].apply(extract_keywords)



# Display the Positive_Review, Important_Tags, and Keywords columns
quality_check = test[['Hotel_Name', 'Positive_Review', 'Keywords']]
quality_check

/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/10415401.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Keywords'] = test['Positive_Review'].apply(extract_keywords)


,Hotel_Name,Positive_Review,Keywords
1,Hotel Arena,No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam,"[great hotel, original offer, new payment, lacking bit, vegetarian anyone, sandwich wrap, great option, minor Aside, fantastic things]"
2,Hotel Arena,Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back,"[good Location, ok staff, nice range]"
4,Hotel Arena,Amazing location and building Romantic setting,[Romantic building]
5,Hotel Arena,Good restaurant with modern design great chill out place Great park nearby the hotel and awesome main stairs,"[modern restaurant, great design, awesome hotel]"
6,Hotel Arena,The room is spacious and bright The hotel is located in a quiet and beautiful park,"[spacious room, quiet hotel]"
...,...,...,...
1030,Apex Temple Court Hotel,Given special permission to use hotel parking alongside blue badge holding parents many thanks for this convenience,"[alongside hotel, many parents]"
1031,Apex Temple Court Hotel,Room was very spacious and clean with all the facilities that you need Bed was very comfortable as well The surroundings is very quiet as the hotel has a innercourt and is not directly situated on a busy road,"[spacious Room, comfortable Bed, quiet surroundings, busy innercourt]"
1032,Apex Temple Court Hotel,The bed really confortable Location is good walking distance to tube and train,"[confortable bed, good Location]"
1033,Apex Temple Court Hotel,Room was gorgeous and very clean Lovely big bed High quality furnishings and technology and air con as well which was an unexpected bonus,"[gorgeous Room, big Lovely, unexpected con]"


In [135]:
# Function to extract the noun from a keyword combo of adjective + noun
def extract_noun(keyword):
    tokens = nltk.word_tokenize(keyword)
    tagged_tokens = nltk.pos_tag(tokens)
    nouns = [word for word, pos in tagged_tokens if pos.startswith('NN')]
    return nouns[0] if nouns else None

# Apply the extract_noun function to the Keywords column and store the result in the Tags column
quality_check['Tags'] = quality_check['Keywords'].apply(lambda keywords: list(set([extract_noun(keyword) for keyword in keywords])))

# Display the updated quality_check dataframe
quality_check

/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/3254981003.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quality_check['Tags'] = quality_check['Keywords'].apply(lambda keywords: list(set([extract_noun(keyword) for keyword in keywords])))


,Hotel_Name,Positive_Review,Keywords,Tags
1,Hotel Arena,No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam,"[great hotel, original offer, new payment, lacking bit, vegetarian anyone, sandwich wrap, great option, minor Aside, fantastic things]","[wrap, payment, Aside, offer, anyone, option, things, bit, hotel]"
2,Hotel Arena,Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back,"[good Location, ok staff, nice range]","[range, Location, staff]"
4,Hotel Arena,Amazing location and building Romantic setting,[Romantic building],[building]
5,Hotel Arena,Good restaurant with modern design great chill out place Great park nearby the hotel and awesome main stairs,"[modern restaurant, great design, awesome hotel]","[hotel, design, restaurant]"
6,Hotel Arena,The room is spacious and bright The hotel is located in a quiet and beautiful park,"[spacious room, quiet hotel]","[room, hotel]"
...,...,...,...,...
1030,Apex Temple Court Hotel,Given special permission to use hotel parking alongside blue badge holding parents many thanks for this convenience,"[alongside hotel, many parents]","[parents, hotel]"
1031,Apex Temple Court Hotel,Room was very spacious and clean with all the facilities that you need Bed was very comfortable as well The surroundings is very quiet as the hotel has a innercourt and is not directly situated on a busy road,"[spacious Room, comfortable Bed, quiet surroundings, busy innercourt]","[surroundings, Room, Bed, innercourt]"
1032,Apex Temple Court Hotel,The bed really confortable Location is good walking distance to tube and train,"[confortable bed, good Location]","[bed, Location]"
1033,Apex Temple Court Hotel,Room was gorgeous and very clean Lovely big bed High quality furnishings and technology and air con as well which was an unexpected bonus,"[gorgeous Room, big Lovely, unexpected con]","[Room, None, con]"


In [155]:
quality_check['Tags'] = quality_check['Tags'].apply(lambda tags: [tag.capitalize() if tag is not None else None for tag in tags])
quality_check.head()


/var/folders/9t/lbrh9vh9073bflym9nbwbr600000gn/T/ipykernel_28863/1006657741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quality_check['Tags'] = quality_check['Tags'].apply(lambda tags: [tag.capitalize() if tag is not None else None for tag in tags])


,Hotel_Name,Positive_Review,Keywords,Tags
1,Hotel Arena,No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam,"[great hotel, original offer, new payment, lacking bit, vegetarian anyone, sandwich wrap, great option, minor Aside, fantastic things]","[Wrap, Payment, Aside, Offer, Anyone, Option, Things, Bit, Hotel]"
2,Hotel Arena,Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back,"[good Location, ok staff, nice range]","[Range, Location, Staff]"
4,Hotel Arena,Amazing location and building Romantic setting,[Romantic building],[Building]
5,Hotel Arena,Good restaurant with modern design great chill out place Great park nearby the hotel and awesome main stairs,"[modern restaurant, great design, awesome hotel]","[Hotel, Design, Restaurant]"
6,Hotel Arena,The room is spacious and bright The hotel is located in a quiet and beautiful park,"[spacious room, quiet hotel]","[Room, Hotel]"


In [153]:

from ipywidgets import interact, Dropdown

def generate_word_cloud():
    # Create a dropdown menu to select the hotel name
    hotel_dropdown = Dropdown(options=table2['Hotel_Name'].unique(), description='Hotel Name:')

    # Define a function to generate the word cloud
    def generate_cloud(hotel_name):
        # Filter the dataframe based on the hotel name
        hotel_df = table2[table2['Hotel_Name'] == hotel_name]

        # Combine all the positive tags into a single string
        tags = hotel_df['Tags'].sum()
        tags_text = ' '.join(tag for tag in tags if tag is not None)

        # Create a word cloud object
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(tags_text)

        # Generate the title for the word cloud
        title = f"The Top Tags of {hotel_name}"

        # Display the word cloud with the title
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(title)
        plt.axis('off')
        plt.show()

    # Interact the dropdown menu with the generate_cloud function
    interact(generate_cloud, hotel_name=hotel_dropdown)


In [154]:
generate_word_cloud()

interactive(children=(Dropdown(description='Hotel Name:', options=('Hotel Arena', 'Apex Temple Court Hotel', '…

To try:

1. extract info from non-noun (e.g. "well located" -> location)

2. bag noun (e.g. "bus", "tube" -> transportation)

3. figure out what "Word embedding" & "Sentiment analysis" would do for us